In [ ]:
import random

# Scoring parameters
MATCH_SCORE = 1
MISMATCH_SCORE = -1
GAP_PENALTY = 0

# Function to calculate the score of an alignment for multiple sequences
def calculate_alignment_score(sequences):
    score = 0
    length = len(sequences[0])

    for i in range(length):
        column = [seq[i] for seq in sequences]
        num_gaps = column.count('-')

        if num_gaps == len(column):
            continue  # All gaps, no score change

        unique_chars = set(column)
        if len(unique_chars) == 1:
            score += MATCH_SCORE  # All match
        elif '-' in unique_chars:
            score += GAP_PENALTY  # Count for gaps
        else:
            score += MISMATCH_SCORE  # Mismatch

    return score

# Function to perform crossover between multiple sequences
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# Function to mutate a sequence
def mutate(sequence, mutation_rate=0.01):
    mutated = []
    for char in sequence:
        if random.random() < mutation_rate:
            mutated.append(random.choice(['A', 'C', 'G', 'T', '-']))
        else:
            mutated.append(char)
    return ''.join(mutated)

# Generate initial population
def generate_population(size, seq_length, num_sequences):
    population = []
    for _ in range(size):
        individual = [''.join(random.choice(['A', 'C', 'G', 'T', '-']) for _ in range(seq_length)) for _ in range(num_sequences)]
        population.append(individual)
    return population

# Genetic Algorithm for multiple sequence alignment
def genetic_multiple_alignment(sequences, population_size=100, generations=50):
    seq_length = len(sequences[0])
    num_sequences = len(sequences)

    # Generate initial population
    population = generate_population(population_size, seq_length, num_sequences)

    for generation in range(generations):
        # Evaluate fitness of the population
        scores = [(individual, calculate_alignment_score(individual)) for individual in population]
        scores.sort(key=lambda x: x[1], reverse=True)

        # Select the top individuals
        selected = [ind for ind, score in scores[:population_size // 2]]

        # Create a new population
        next_population = selected[:]

        # Crossover
        while len(next_population) < population_size:
            parent1, parent2 = random.sample(selected, 2)
            children = crossover(parent1, parent2)
            next_population.extend(children)

        # Mutation
        next_population = [mutate(individual) for individual in next_population]

        # Update population
        population = next_population

    # Get the best alignment from the final population
    best_individual = max(population, key=lambda ind: calculate_alignment_score(ind))
    return best_individual, calculate_alignment_score(best_individual)

# Usage
if __name__ == "__main__":
    sequences = [
        "ACGATAGAGAGT",
        "ACGATAGAGAGT"
    ]
    best_alignment, best_score = genetic_multiple_alignment(sequences)
    print(f"Best Alignment:\n{best_alignment}")
    print(f"Score: {best_score}")